In [1]:
#%pip install lxml
import os
import s3fs
import pandas as pd
import xml.etree.ElementTree as ET
#from lxml import etree

os.environ["AWS_ACCESS_KEY_ID"] = 'GMHBU5SUKVUN1A81Y8UY'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'o+beqI9AemFNrUL4c0OSTxDbOhg4llbcwl0ScqEm'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJHTUhCVTVTVUtWVU4xQTgxWThVWSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzY5NjEyMTg2LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzcwNzk4MzU4LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc3MDE5MzU1NywiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDplNzAyY2NmOS05N2FjLTNkMDgtNDk0YS1iZWQ1M2EzMzYyMzQiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6Ijc4ODAyYmFkLTFjZTEtODFkOC1mYjI4LTQwNWY4MWQ0ZGVmNSIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.UhD0r4MQ2qUkefOj6PjF-awWui47amNDnE3PLOZnuc30X6GTsfDb4N9Aqm2acDms4ez0btnYSOi6_to_-wQlsw'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])
with fs.open("s3://lab/conf_net.parquet") as f:
   conf = pd.read_parquet(f)
with fs.open("s3://lab/art_net.parquet") as f:
    art = pd.read_parquet(f)



In [ ]:
df = pd.concat(
    [art.explode("authors")["authors"],
     conf.explode("authors")["authors"]],
    ignore_index=True
)

df = df.apply(pd.Series)[['name', 'orcid']]
df

In [ ]:
with fs.open(f"s3://lab/orcids.parquet", "wb") as f:
            df.to_parquet(f, index=False)

,title,journal,year,authors,dblp_uri,doi
0,Auswirkung der Digitalisierung auf die Systeml...,Elektrotech. Informationstechnik,2019,"{'name': 'Kai Schlabitz', 'orcid': None}",journals/ei/Schlabitz19,https://doi.org/10.1007/s00502-019-0687-y
1,EMF-Personenschutz: Neue Aspekte in der numeri...,Elektrotech. Informationstechnik,2020,"{'name': 'Richard berbacher', 'orcid': None}",journals/ei/UberbacherC20,https://doi.org/10.1007/s00502-020-00791-z
1,EMF-Personenschutz: Neue Aspekte in der numeri...,Elektrotech. Informationstechnik,2020,"{'name': 'Stefan Cecil', 'orcid': None}",journals/ei/UberbacherC20,https://doi.org/10.1007/s00502-020-00791-z
2,Zur Genesis der Forschungsstelle fr Integriert...,Elektrotech. Informationstechnik,2022,"{'name': 'Herbert Mang', 'orcid': None}",journals/ei/Mang22,https://doi.org/10.1007/s00502-022-01049-6
3,100 % erneuerbare Energie fr sterreichs Indust...,Elektrotech. Informationstechnik,2021,"{'name': 'Sophie Knttner', 'orcid': None}",journals/ei/KnottnerGDD21,https://doi.org/10.1007/s00502-021-00953-7
...,...,...,...,...,...,...
3992220,Analysis of projected hydrological behavior of...,Hydrology and Earth System Sciences,2012,"{'name': 'Rita Ley', 'orcid': None}",persons/CasperGGGHLR12,https://doi.org/10.5194/hess-16-409-2012
3992220,Analysis of projected hydrological behavior of...,Hydrology and Earth System Sciences,2012,"{'name': 'Andreas Rock', 'orcid': None}",persons/CasperGGGHLR12,https://doi.org/10.5194/hess-16-409-2012
3992221,Common Subexpression Identification in General...,"Technical Rep. UKSC 0060, IBM United Kingdom S...",1974,"{'name': 'Patrick A. V. Hall', 'orcid': None}",persons/Hall74,None
3992222,Interactive Support for Non-Programmers: The R...,"Research Report / RJ / IBM / San Jose, California",1974,"{'name': 'E. F. Codd', 'orcid': None}",persons/CoddD74,None
